## Importations

In [1]:
import numpy as np
import pandas as pd
import itertools as it
import scipy
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer

# Global variables

In [2]:
# Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users.
# Average computation time without optimization : 10s/user --> 6100s --> less than 2 hours
# Full: 27,000,000 ratings and 1,100,000 tag applications applied to 58,000 movies by 280,000 users.
# Average computation time without optimization : 

In [3]:
PATH = "../data/"
DATASET_TYPE = "small"
TRAIN_SIZE = 0.8
MINIMUM_SEEN_MOVIES = 1
MINIMUM_SEEN_USERS = 20

## Loading data

In [4]:
def load_data(path, dataset_type="small"):
	if dataset_type == "small":
		movies = pd.read_csv(path + "ml-latest-small/movies.csv")
		ratings = pd.read_csv(path + "ml-latest-small/ratings.csv")
	elif dataset_type == "full":
		movies = pd.read_csv(path + "ml-latest/movies.csv")
		ratings = pd.read_csv(path + "ml-latest/ratings.csv")
	return movies, ratings

In [5]:
movies, ratings = load_data(PATH, DATASET_TYPE)

# Preprocessing data

In [6]:
## Delete films seen only by a certain number of users
def get_id_delete_solo_films(data, threshold,nom_colonne) :
    '''
    data -> movies ou ratings ( dataframe qui contient une colonne movieId )
    '''
    list_key_values = np.array(list(Counter(data[nom_colonne].values).items()))
    key,values = list_key_values[:,0],list_key_values[:,1]
    id_delete =  np.where(values < threshold)[0]
    return key[id_delete]

def delete_solo_films(data,id_delete,nom_colonne) :
    '''
    data -> movies ou ratings ( dataframe qui contient une colonne movieId )
    '''
    array_movieId = data[nom_colonne].values
    ind = [i for i in range(len(array_movieId)) if array_movieId[i] not in id_delete ]
    return data.iloc[ind]


## Building ratings and movies dataframe which both contains the same movieId
def clear_dataset(movies, ratings):

	id_delete = get_id_delete_solo_films(ratings, MINIMUM_SEEN_MOVIES,'movieId')
	ratings = delete_solo_films(ratings,id_delete,'movieId')
	movies = delete_solo_films(movies,id_delete,'movieId')
	id_delete = get_id_delete_solo_films(ratings, MINIMUM_SEEN_USERS,'userId')
	ratings = delete_solo_films(ratings,id_delete,'userId')
	
	list_movieId = list(set(movies["movieId"].values).intersection(set(ratings["movieId"].values)))
	movies_old = movies['movieId'].values
	l = []
	for i in range(len(movies_old)):
		if movies_old[i] in list_movieId:
			l.append(i)
	movies = movies.iloc[l,:]

	a = sorted(list(list_movieId))
	b = range(len(a))
	d = dict(zip(a,b))
	movies = movies.replace({'movieId' : d})

	a = sorted(list(list_movieId))
	b = range(len(a))
	d = dict(zip(a,b))
	ratings = ratings.replace({'movieId' : d})
    
	ratings.index = range(len(ratings))
	movies.index = range(len(movies))

	return movies, ratings	
	
## Building one hot encoded genres in movies dataframe
def one_hot_encode_genres(movies):
	tmp = []
	for elt in movies["genres"]:
		tmp.append(elt.split("|"))
	movies["genres"] = tmp
	mlb = MultiLabelBinarizer(sparse_output=True)
	movies = movies.join(
				pd.DataFrame.sparse.from_spmatrix(
												mlb.fit_transform(movies.pop('genres')),
												index=movies.index,
												columns=mlb.classes_))
	return movies

## Cleaning ratings datagrame
def preprocess_ratings(ratings):
	ratings = ratings.drop(columns=["timestamp"])
	ratings['userId'] = ratings['userId'].to_numpy() - 1 # car pas de user 0
	return ratings

## Split for computing metrics on test later
def split_set(userId, train_size, ratings):
	rating_user = ratings[ratings["userId"] == userId]
	train_rating_user, test_rating_user = rating_user.to_numpy()[:int(train_size*len(rating_user))], rating_user.to_numpy()[int(train_size*len(rating_user)):]
	return train_rating_user, test_rating_user

## Get informations on users watched/unwatched movies...
def get_infos_user(userId):
	watched_user = set(ratings[ratings["userId"] == userId]["movieId"])
	watched_all = set(ratings['movieId'])
	unwatched_user = list(watched_all.difference(watched_user))
	return watched_user, watched_all, unwatched_user

In [7]:
movies, ratings = clear_dataset(movies, ratings)
movies = one_hot_encode_genres(movies)
ratings = preprocess_ratings(ratings)

# Building matrix

In [8]:
# Building a sparse matrix which contains the triple (u_k, m_i, r_ki)
# def build_sparse_matrix_triples(ratings):
# 	ratings_sparse = scipy.sparse.csr_matrix(ratings.values)
# 	return ratings_sparse

## Building a matrix M = (n_movies, n_movies) which contains the number of users who'se seen m_i and m_j
def build_M_matrix(ratings, train_size):
	data_dict = dict()
	train_rating_user_list = []
	test_rating_user_list = []
	for userId in tqdm(set(ratings["userId"])):
		train_rating_user, test_rating_user = split_set(userId, train_size, ratings)
		train_rating_user_list.append(np.array(train_rating_user))
		test_rating_user_list.append(np.array(test_rating_user))
		iterator = it.combinations(train_rating_user[:,1], 2)
		for x, y in iterator:
			data_dict[(x,y)] = data_dict.get((x,y), 0.) + 1.
			data_dict[(y,x)] = data_dict.get((y,x), 0.) + 1.
		iterator = it.combinations(test_rating_user[:,1], 2)
		for x, y in iterator:
			# We need to ignore the test movies
			data_dict[(x,y)] = 0
			data_dict[(y,x)] = 0
	keys = np.array(list(data_dict.keys())).astype(int)
	values = np.array(list(data_dict.values())).astype(float)
	M_coo = scipy.sparse.coo_matrix((values, (keys[:,0], keys[:,1])))
	M_csr = M_coo.tocsr()
	M_norm = M_csr
	return M_norm, train_rating_user_list, test_rating_user_list

## Computing probabilites of genres P_ig
def build_P_ig(movies):
	sum_ = movies[[i for i in movies.columns if i != "movieId" and i != "title"]].to_numpy().sum(axis=0).astype(int)
	P_ig = sum_ / sum(sum_)
	return P_ig.reshape(-1, 1)

## Initialisation of R_uk before iterative algorithm
def init_R_uk(movies):
	n_genres = len(movies.columns) - 2
	n_movies = len(movies)
	r = 1/(n_movies*n_genres)
	R_uk = np.full((n_movies, n_genres), r)
	return R_uk

## Computing F_ig for each user
def build_F_ig(R_uk, P_ig):
	F_ig = np.sum(R_uk, axis=1).reshape(-1,1) @ P_ig.reshape(1,-1)
	return F_ig

## Matrix user X movie
def build_ratings_matrix(ratings):
	values = ratings["rating"]
	rows = ratings["userId"]
	cols = ratings["movieId"]
	M_coo = scipy.sparse.coo_matrix((values, (rows, cols)))
	M_csr = M_coo.tocsr()
	return M_csr

## Build I_uk for each user
def build_I_uk(tmp_M, id_user, P_ig):
# 	print(tmp_M[id_user,:].T.shape)
	I_uk = tmp_M[id_user,:].T @ P_ig.reshape(1,-1)
	I_uk = I_uk / I_uk.sum(axis=0).T
	return I_uk

## Init the matrix needed before running the iterative algorithm
def init(movies, ratings, train_size):
	print("Init R_uk...")
	R_uk = init_R_uk(movies)
	print(R_uk.shape)
	print("Building P_ig...")
	tmp_M = build_ratings_matrix(ratings)
	P_ig = build_P_ig(movies)
	print(P_ig.shape)
	print("Building M_csr...")
	M_csr, train_rating_user_list, test_rating_user_list = build_M_matrix(ratings, train_size)
	print(M_csr.shape)
	return R_uk, tmp_M, P_ig, M_csr, np.array(train_rating_user_list, dtype=object), np.array(test_rating_user_list, dtype=object)

In [9]:
R_uk, tmp_M, P_ig, M_csr, train_rating_user_list, test_rating_user_list = init(movies, ratings, TRAIN_SIZE)

Init R_uk...
(9724, 20)
Building P_ig...
(20, 1)
Building M_csr...


100%|██████████| 610/610 [00:33<00:00, 18.00it/s]


(9724, 9724)


## Run the algorithm

In [12]:
## Compute TR_ki for a specific user
def compute_TR_ki(id_user, R_uk, tmp_M, P_ig, M_csr, d, alpha, iter_max):
	I_uk = build_I_uk(tmp_M, id_user, P_ig)
	for _ in range(iter_max):
		F_ig = build_F_ig(R_uk, P_ig)
		R_uk = d * alpha * M_csr @ R_uk + d * (1-alpha) * M_csr @ F_ig + (1-d) * I_uk

		# This part is useful if you want to normalize + break if converge
		# R_uk = (R_uk / R_uk.sum(axis=1)).T # Normalization isn't working
		#     print(np.abs(np.sum(R_uk - R_uk_old)))
		#     if np.abs(np.sum(R_uk - R_uk_old)) < eps :
		#         print(i)
		#         break
		# R_uk_old = R_uk.copy()
	
	TR_ki = np.array(R_uk @ P_ig) # It returns a np.mat object which can't be reduced to dimension 1
	return TR_ki.reshape(-1)





import multiprocessing
from functools import partial

## Compute TR_ki for all users
def iterative_TR_ki(n_user, R_uk, tmp_M, P_ig, M_csr, d=0.15, alpha=0.1, iter_max=5):
	print("Computing TR_ki for all users...")
	TR_ki_all_user = []

	# pool_obj = multiprocessing.Pool(processes=4)
	# prod_x = partial(compute_TR_ki, R_uk=R_uk, tmp_M=tmp_M, P_ig=P_ig, M_csr=M_csr, d=d, alpha=alpha, iter_max=iter_max)
	# pool_obj.map(prod_x, range(n_user))

	for id_user in tqdm(range(n_user)):
		TR_ki_all_user.append(compute_TR_ki(id_user, R_uk, tmp_M, P_ig, M_csr, d, alpha, iter_max))

	return np.array(TR_ki_all_user)

In [13]:
# n_user = len(np.unique(ratings["userId"]))
n_user = 3 ## Use when testing

TR_ki_all_user = iterative_TR_ki(n_user, R_uk, tmp_M, P_ig, M_csr, d=0.15, alpha=0.1, iter_max=15)
# Parallel(n_jobs=2)(delayed(iterative_TR_ki)(n_user, R_uk, tmp_M, P_ig, M_csr, d=0.15, alpha=0.1, iter_max=15))

Computing TR_ki for all users...


## Running some test for a test user

In [ ]:
## Returns the recommandation for the users
def sort_by_best_movie(TR_ki_all_user):
	sorted_movies_all_user = np.zeros_like(TR_ki_all_user)
	for i in range(len(TR_ki_all_user)):
		sorted_movies_all_user[i,:] = np.argsort(TR_ki_all_user[i,:])[::-1]
	return sorted_movies_all_user

In [ ]:
test_user_id = 1

print("TR_ki_all_user shape:", TR_ki_all_user.shape)
print("test_rating_user_list shape:", test_rating_user_list.shape)
print("TR_ki for test user :\n", TR_ki_all_user[test_user_id, :10])

In [ ]:
sorted_movies_all_user = sort_by_best_movie(TR_ki_all_user)
print("sorted_movies_all_user shape:", sorted_movies_all_user.shape)
print("Sorted best movies recommandation for test user :\n", sorted_movies_all_user[test_user_id,:10])

## Running some test for the DOA metrics before computing it on whole user list

In [ ]:
## Computes DOA score for a specific user
def compute_doa_score(TR_ki, test_rating_user, unwatched_user):
	score = 0
	for m_i in test_rating_user:
		for m_j in unwatched_user:
			if TR_ki[int(m_i)] > TR_ki[int(m_j)]:
				score += 1
	return score / (len(test_rating_user) * len(unwatched_user))

## Computes DOA for the whole user list
def compute_all_doa(TR_ki_all_user, test_rating_user_list):
	score = 0
	n_user = TR_ki_all_user.shape[0]
	for user_id in range(n_user):
		tmp = test_rating_user_list[user_id]
		test_films = tmp[:,1]
		TR_ki = TR_ki_all_user[user_id]
		_, _, unwatched_films = get_infos_user(user_id)
		score += compute_doa_score(TR_ki, test_films, unwatched_films)
	return score / n_user

def test_doa_user(n_user, n_film_test):
	for test_user_id in range(n_user):
		print("---------- Testing user number", test_user_id, "----------")
		test = test_rating_user_list[test_user_id]
		test_films = test[:,1]
		TR_ki = TR_ki_all_user[test_user_id]
		_, _, unwatched_films = get_infos_user(test_user_id)
		print("Some movies from test set:")
		print(test_films[:n_film_test])
		print("Some movies from unwatched set:")
		print(unwatched_films[:n_film_test])
		print("Some score for test set (movies watched by user):")
		for i in test_films[:n_film_test]:
			print(TR_ki[int(i)])
		print("Some score for unwatched movies:")
		for i in unwatched_films[:n_film_test]:
			print(TR_ki[int(i)])
		print("Total DOA score for this user :", compute_doa_score(TR_ki, test_films, unwatched_films))

In [ ]:
test_doa_user(n_user=1, n_film_test=4)

## Computing final DOA metric for the whole dataset

In [ ]:
print("DOA for all users :", compute_all_doa(TR_ki_all_user, test_rating_user_list))

## Building a dataset to do an experiment campaign

In [ ]:
def build_experimental_dataset(movies, TR_ki_all_user, train_rating_user_list, test_rating_user_list, n_user, n_film_test):
	dataset = []
	dict_id_title = dict(zip(list(movies['movieId']),list(movies['title'])))
	for userId in range(n_user) :
		train_rating_user = train_rating_user_list[0][:,1].astype(int)
		test_rating_user = test_rating_user_list[0][:,1].astype(int)
		_, _, unwatched_user = get_infos_user(userId)
		unwatched_user = np.append(test_rating_user, unwatched_user)

		ind_sort =  np.argsort(TR_ki_all_user[userId,:])[::-1]

		watched_user_final = []
		unwatched_user_final = []

		for ind in ind_sort :

			if ind in train_rating_user:
				watched_user_final.append(ind)
			else : #ind in unwatched_user_final
				unwatched_user_final.append(ind)

		watched_user_best = watched_user_final[:n_film_test]
		unwatched_user_best = unwatched_user_final[:n_film_test]
		unwatched_user_rand = np.random.choice(np.array(unwatched_user_final[n_film_test+1:]), size=n_film_test)

		watched_user_best_dict = { str(movieId) : dict_id_title[movieId] for movieId in watched_user_best } 
		unwatched_user_best_dict = { str(movieId) : dict_id_title[movieId] for movieId in unwatched_user_best } 
		unwatched_user_rand_dict = { str(movieId) : dict_id_title[movieId] for movieId in unwatched_user_rand }

		dataset.append((watched_user_best_dict,unwatched_user_best_dict,unwatched_user_rand_dict))

	return np.array(dataset)

In [ ]:
dataset = build_experimental_dataset(movies, TR_ki_all_user, train_rating_user_list, test_rating_user_list, n_user, n_film_test=5)

In [ ]:
dataset.shape

## Testing our experiment dataset

In [ ]:
userId = 0
dataset[userId]

In [ ]:
n_user_experiment = 10
print(dataset[:n_user_experiment])

In [ ]:
np.array(dataset[:n_user_experiment]).shape

## Explication & Dimension

In [ ]:
# uk -> user k 
# ig -> movies i, genre g 
# R_uk -> movie,genre pour user uk
# P_ig -> désigne la probabilité avec laquelle l'item i appartient au genre g
# M -> matrice correlation
# F -> 
# I_uk -> 
# r_ki -> 

#R -> n_user X n_movies X n_genres
#I -> n_user X n_movies X n_genres

#M -> n_movies X n_movies
#F_ig -> n_movies X n_genres
#I_uk -> n_movies X n_genres

# Split train test sur les users????????
# Entrainement sur le train 
# Test a la fin comparaison NDCG entre tri du ranking et notes de l'utilisateur 

# Split train test sur les ratings par users (on ignore les ratings test pour le train)
# Entrainement sur le train (il reste le meme, on a juste caché des notes)
# Test a la fin comparaison NDCG entre tri du ranking (donné par l'algo grace au train) et notes de l'utilisateur (qu'on a garder en test)
# Test avec le DOA entre les films (qu'on a cache) et les films unwatched ?

# On prend pour un utilisateur sa liste de film + ses notes
# On montre cette liste a des nouveaux utilisateurs puis on leur demande parmis les films que notre algo sort quel est celui qui devrait etre recommandé
# On compare les resultats avec notre ranking reel apres train
# Test de student 